## Testing of the Graph Environment

Imports

In [1]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym
import pickle
from datetime import datetime

import ray

import warnings
warnings.filterwarnings('ignore')


In [2]:
#from ray.rllib.agents.dqn import DQNTrainer, DEFAULT_CONFIG, APEX_DEFAULT_CONFIG
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

Import the Environment

In [3]:
sys.path.insert(0,"")

from ManhattanGraph import ManhattanGraph
from gym_graphenv.envs.GraphworldManhattan import GraphEnv

DB: getAvailableTrips() Execution time: 10.162337064743042 seconds
['id0000569', 'id0450655']
DB: getRouteFromTrip() Execution time: 0.0010061264038085938 seconds
([42449991, 42429690, 42445236, 42436486, 42438800, 42439561, 42430063, 42450009, 42446472], ['2016-01-03 19:51:38', '2016-01-03 19:51:53', '2016-01-03 19:52:08', '2016-01-03 19:52:24', '2016-01-03 19:52:40', '2016-01-03 19:52:55', '2016-01-03 19:53:10', '2016-01-03 19:53:25', '2016-01-03 19:53:40'])


In [4]:
env_config = {'pickup_hub_index':2,
        'delivery_hub_index':3,
        'pickup_timestamp': datetime(2016, 6, 14, 22, 0, 0),
        'delivery_timestamp': datetime(2016, 6, 14, 1, 0, 0) 
}

with open('env_config.pkl', 'wb') as f:
    pickle.dump(env_config, f)

In [5]:
env=GraphEnv()


Run random steps without Ray Trainer

In [6]:
def run_one_episode (env):
    env.reset()
    print("reset done")
    sum_reward = 0
    for i in range(30):
        action = env.action_space.sample()
        #print(action)
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("DELIVERY DONE! sum_reward: ",sum_reward)
            break

        print("sum_reward: ",sum_reward)
        # print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

for i in range(1): 
    run_one_episode (env)

Reset with config
final hub: 3
current position: 2
DB: getAvailableTrips() Execution time: 11.57048225402832 seconds
reset done
Time Ride: 0.007216215133666992
DB: getAvailableTrips() Execution time: 10.44108510017395 seconds
Time Adj Matrix: 0.10086584091186523
2 -> 43 50.0
Step() Execution time: 10.664422750473022 seconds
sum_reward:  -50.0
Time Ride: 0.002310037612915039
DB: getAvailableTrips() Execution time: 10.374144077301025 seconds
DB: getRouteFromTrip() Execution time: 0.004598855972290039 seconds
Time Adj Matrix: 0.10106301307678223
43 -> 20 50.0
Step() Execution time: 10.607732057571411 seconds
sum_reward:  -100.0
Time Ride: 0.0004298686981201172
DB: getAvailableTrips() Execution time: 10.125295877456665 seconds
DB: getRouteFromTrip() Execution time: 0.0030357837677001953 seconds
Time Adj Matrix: 0.09858298301696777
20 -> 39 50.0
Step() Execution time: 10.341443061828613 seconds
sum_reward:  -150.0
Time Ride: 0.034876108169555664
DB: getAvailableTrips() Execution time: 10.24

KeyboardInterrupt: 

Initialize Ray

In [7]:
ray.init()

2022-05-12 10:27:31,205	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-05-12_10-27-24_713240_43563/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-05-12_10-27-24_713240_43563/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-05-12_10-27-24_713240_43563',
 'metrics_export_port': 54898,
 'gcs_address': '127.0.0.1:53565',
 'address': '127.0.0.1:53565',
 'node_id': '0e7617ae347f6ab4580d48250cf6b8edeb28af6f92a7a7c7aa5c8c8f'}

Set DQN trainer configuration

In [8]:
dqn_config = DEFAULT_CONFIG.copy()
dqn_config['num_workers'] = 1
dqn_config["train_batch_size"] = 400
dqn_config["gamma"] = 0.95
dqn_config["n_step"] = 10
dqn_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#dqn_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

Set DQN - Rainbow configuration

In [9]:
rainbow_config = DEFAULT_CONFIG.copy()

rainbow_config = DEFAULT_CONFIG.copy()
rainbow_config['num_workers'] = 1
rainbow_config["train_batch_size"] = 400
rainbow_config["gamma"] = 0.95
rainbow_config["n_step"] = 10
rainbow_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#rainbow_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

#rainbow parameters

# N-step Q learning
rainbow_config["n_step"]= 5 #[between 1 and 10]
# Whether to use noisy network
rainbow_config["noisy"] = True
# Number of atoms for representing the distribution of return. When
# this is greater than 1, distributional Q-learning is used.
# the discrete supports are bounded by v_min and v_max
rainbow_config["num_atoms"] =2 #[more than 1]
rainbow_config["v_min"] =-100
rainbow_config["v_max"]=100 # (set v_min and v_max according to your expected range of returns).

Set DQN - Ape-X configuration 

In [10]:
#apex_config=APEX_DEFAULT_CONFIG.copy()

Initialize the DQN trainer, Rainbow DQN and DQN Ape-X

In [8]:
ppo_trainer=PPOTrainer(DEFAULT_CONFIG,GraphEnv)

2022-05-12 10:27:46,846	INFO trainer.py:2140 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-05-12 10:27:46,856	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-05-12 10:27:46,863	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=43657) DB: getAvailableTrips() Execution time: 34.01478290557861 seconds
(RolloutWorker pid=43657) ['id0000569', 'id0450655']
(RolloutWorker pid=43657) DB: getRouteFromTrip() Execution time: 0.015662193298339844 seconds
(RolloutWorker pid=43657) ([42449991, 42429690, 42445236, 42436486, 42438800, 42439561, 42430063, 42450009, 42446472], ['2016-01-03 19:51:38', '2016-01-03 19:51:53', '2016-01-03 19:52:08', '2016-01-03 19:52:24', '2016-01-03 19:52:40', '2016-01-03 19:52:55', '2016-01-03 19:53:10', '2016-01-03 19:53:25', '2016-01-03 19:53:40'])
(RolloutWorker pid=43658) DB: getAvailableTrips() Execution time: 34.01480197906494 seconds
(RolloutWorker pid=43658) ['id0000569', 'id0450655']
(RolloutWorker pid=43658) DB: getRouteFromTrip() Execution time: 0.015657901763916016 seconds
(RolloutWorker pid=43658) ([42449991, 42429690, 42445236, 42436486, 42438800, 42439561, 42430063, 42450009, 42446472], ['2016-01-03 19:51:38', '2016-01-03 19:51:53', '2016-01-03 19:52:08', '2016

(RolloutWorker pid=43657) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=43657) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=43657) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=43658) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=43658) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=43658) UserWarning: The `get_nearest

(RolloutWorker pid=43657) Reset with config
(RolloutWorker pid=43657) final hub: 3
(RolloutWorker pid=43657) current position: 2
(RolloutWorker pid=43658) Reset with config
(RolloutWorker pid=43658) final hub: 3
(RolloutWorker pid=43658) current position: 2


In [ ]:
trainer_dqn = DQNTrainer(dqn_config,GraphEnv)

In [ ]:
trainer_rainbow=DQNTrainer(rainbow_config,GraphEnv )


In [ ]:
#trainer_apex=DQNTrainer(apex_config,GraphEnv )

Define the path where the results of the trainer should be saved

In [ ]:
checkpoint_root = "tmp/dqn/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Run the trainer

In [ ]:
results = []
episode_data = []
episode_json = []
n_iter=10

for n in range(n_iter):
    result = ppo_trainer.train()
    results.append(result)
    print("TEST",n)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = ppo_trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

(RolloutWorker pid=39477) Reset with config
(RolloutWorker pid=39477) final hub: 3
(RolloutWorker pid=39477) current position: 2
(RolloutWorker pid=39478) Reset with config
(RolloutWorker pid=39478) final hub: 3
(RolloutWorker pid=39478) current position: 2


(RolloutWorker pid=39477) FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
(RolloutWorker pid=39478) FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.


(RolloutWorker pid=39477) action ==  54  New Position 54
(RolloutWorker pid=39478) action ==  35  New Position 35
(RolloutWorker pid=39477) 2 -> 54 50.0
(RolloutWorker pid=39477) Step() Execution time: 0.3731091022491455 seconds
(RolloutWorker pid=39478) 2 -> 35 50.0
(RolloutWorker pid=39478) Step() Execution time: 0.3792250156402588 seconds
(RolloutWorker pid=39477) action ==  62  New Position 62
(RolloutWorker pid=39478) action ==  38  New Position 38
(RolloutWorker pid=39477) 54 -> 62 50.0
(RolloutWorker pid=39477) Step() Execution time: 0.48279881477355957 seconds
(RolloutWorker pid=39477) action ==  41  New Position 41
(RolloutWorker pid=39478) 35 -> 38 50.0
(RolloutWorker pid=39478) Step() Execution time: 0.4827907085418701 seconds
(RolloutWorker pid=39478) action ==  47  New Position 47
(RolloutWorker pid=39477) 62 -> 41 50.0
(RolloutWorker pid=39477) Step() Execution time: 0.6641361713409424 seconds
(RolloutWorker pid=39477) action ==  18  New Position 18
(RolloutWorker pid=394

KeyboardInterrupt: 

(RolloutWorker pid=39477) 28 -> 36 50.0
(RolloutWorker pid=39477) Step() Execution time: 0.5706348419189453 seconds
(RolloutWorker pid=39477) action ==  30  New Position 30
(RolloutWorker pid=39478) 59 -> 21 50.0
(RolloutWorker pid=39478) Step() Execution time: 0.6509459018707275 seconds
(RolloutWorker pid=39478) action ==  18  New Position 18
(RolloutWorker pid=39478) 21 -> 18 50.0
(RolloutWorker pid=39478) Step() Execution time: 1.0859730243682861 seconds
(RolloutWorker pid=39477) 36 -> 30 50.0
(RolloutWorker pid=39477) Step() Execution time: 1.199338436126709 seconds
(RolloutWorker pid=39478) action ==  11  New Position 11
(RolloutWorker pid=39477) action ==  40  New Position 40
(RolloutWorker pid=39477) 30 -> 40 50.0
(RolloutWorker pid=39477) Step() Execution time: 0.5341320037841797 seconds
(RolloutWorker pid=39478) 18 -> 11 50.0
(RolloutWorker pid=39478) Step() Execution time: 0.6856091022491455 seconds
(RolloutWorker pid=39478) action ==  52  New Position 52
(RolloutWorker pid=3

In [ ]:
results

Testing of the trainer

In [ ]:
trainer.restore(file_name)
env = gym.make("graphworld-v0")
state = env.reset()

In [ ]:
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = trainer_dqn.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == 1:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

In [ ]:
ray.shutdown()